# Import Packages and environmental setup

In [1]:
import numpy as np
import random
import ete3
import torch
torch.manual_seed(0)
import torch.nn as nn
from torch_geometric.data import Data
import itertools
import json
import sys
import time
from tqdm import tqdm
import os
from os import path
sys.path.insert(0, '../')
import gc
import torch_geometric.transforms as T
from torch_geometric.nn.conv import TransformerConv
from torch_geometric.nn import VGAE
from torch_geometric.loader import DataLoader
from torch_geometric.utils import batched_negative_sampling
from ete3 import Tree
gc.collect()

11

# Functions

The internal node will be in the order of 5-7-6, the single tip will always be connected to node 7


In [2]:
# function to convert string to numbers
def convert_string_to_numbers(str, dict):
    ''' str: string to convert
        dict dictionary with the relative ordering of each char'''
            # create a map iterator using a lambda function
    # lambda x -> return dict[x]
    # This return the value for each key in dict based on str
    numbers = map(lambda x: dict[x], str)
    # return an array of int64 numbers
    return np.fromiter(numbers, dtype=np.int64)

In [4]:
# function to create a graph for each 
def construct_single_graph(idx, t):
    ''' idx: the current graph index w.r.t the label
        t: the tree object from ete'''
    # transform the character of amino acid in to numbers for all 5 sequences in this graph
    transformed_x = []
    for i in range(5):
        # get the index of the sequence from the original dataset
        seq_idx = 5*idx + i
        transformed_x.append(convert_string_to_numbers(seq_string[seq_idx][:-1], dict_amino))
        
    # initialize the sequence of 3 internal nodes
    vec_len = len(transformed_x[0])
    internal_node_5 = np.full(vec_len, -1, dtype=np.int64)
    internal_node_6 = np.full(vec_len, -1, dtype=np.int64)
    internal_node_7 = np.full(vec_len, -1, dtype=np.int64)
    
    # Work out the branch distance from the Newick format
    leaf_pair = 0               # The amount of leaf pair so far, max=2
    prev_leaf = False           # Whether the previous leaf in the preorder is a leaf node
    prev_dist = 0               # The distance of the branch coming out of the preivous node in preorder
    dist_array = [0]*8          # The distance for outgoing branch for each node, node 7 will always be 0
    prev_index = -1             # The index of the last leaf node in the preorder
    tot_in_node = 0             # All distance of internal nodes that are unassigned so far
    pending = False             # Some condition for assigning branch length that I don't remember
    preorder=[]                 # The preorder of all leaf nodes
    
    # Traverse through all nodes in preorder, work out the branch distance 
    # There are only 2 possible rooted tree format from ETE, 
    # so 2 if statements that work out all different scenarios
    for node in t.traverse("preorder"):
        if not node.name=='':
            index = int(node.name) - 1
            preorder.append(index)
            dist_array[index] = node.dist
            prev_index = index
            if leaf_pair >= 2:
                tot_in_node += node.dist
                dist_array[index] = tot_in_node
                break
            else:
                if prev_leaf==False:
                    prev_leaf=True
                else:
                    leaf_pair += 1
                    prev_leaf=False
                    if prev_dist != 0:
                        dist_array[leaf_pair+4] = prev_dist
                    else:
                        pending = True
                    tot_in_node-=prev_dist
        else:
            prev_dist = node.dist
            tot_in_node+=node.dist
            if pending==True:
                pending = False
                prev_dist = 0
                tot_in_node -= node.dist
                dist_array[leaf_pair+4] = node.dist
            if prev_leaf==True:
                dist_array[prev_index] += node.dist
                prev_dist = 0
                tot_in_node -= node.dist
    # Set up the adjency Matrix in COO format
    # We find the smaller node number of each side.
    # In this case, the tip with the larger node number is on the left side, thus connect to node 5
    if min(preorder[0], preorder[1]) > min(preorder[2], preorder[3]):
        # change edge value of edge 5 and 6
        # I think this is due to the conditions from the previous part, but I don't remember the details
        # It works though!
        tmp = dist_array[5]
        dist_array[5] = dist_array[6]
        dist_array[6] = tmp
        # Assign edge origin/destination and value
        edge_index = torch.tensor([[preorder[2],5],[5,preorder[2]],[preorder[3],5],[5,preorder[3]],
                                       [5,7],[7,5],[preorder[4],7],[7,preorder[4]],
                                       [6,7],[7,6],[preorder[0],6],[6,preorder[0]],
                                       [preorder[1],6],[6,preorder[1]]], dtype=torch.long)
        edge_attr = [dist_array[preorder[2]], dist_array[preorder[2]], 
                 dist_array[preorder[3]], dist_array[preorder[3]], 
                 dist_array[5], dist_array[5],
                 dist_array[preorder[4]], dist_array[preorder[4]],
                 dist_array[6], dist_array[6],
                 dist_array[preorder[0]], dist_array[preorder[0]],
                 dist_array[preorder[1]], dist_array[preorder[1]]]
        # Assign the value for internal node 5 and 6, based on the 2 leaf node they are connected with
        for j in range(0,vec_len):
            internal_node_5[j] = random.choice([transformed_x[preorder[2]][j],transformed_x[preorder[3]][j]])
            internal_node_6[j] = random.choice([transformed_x[preorder[0]][j],transformed_x[preorder[1]][j]])
    # Same thing, but now the smaller node number is on the left, thus connected with node 5
    else:
        edge_index = torch.tensor([[preorder[0],5],[5,preorder[0]],[preorder[1],5],[5,preorder[1]],
                                       [5,7],[7,5],[preorder[4],7],[7,preorder[4]],
                                       [6,7],[7,6],[preorder[2],6],[6,preorder[2]],
                                       [preorder[3],6],[6,preorder[3]]], dtype=torch.long)
        edge_attr = [dist_array[preorder[0]], dist_array[preorder[0]], 
                 dist_array[preorder[1]], dist_array[preorder[1]], 
                 dist_array[5], dist_array[5],
                 dist_array[preorder[4]], dist_array[preorder[4]],
                 dist_array[6], dist_array[6],
                 dist_array[preorder[2]], dist_array[preorder[2]],
                 dist_array[preorder[3]], dist_array[preorder[3]]]
        for j in range(0,vec_len):
            internal_node_5[j] = random.choice([transformed_x[preorder[0]][j],transformed_x[preorder[1]][j]])
            internal_node_6[j] = random.choice([transformed_x[preorder[2]][j],transformed_x[preorder[3]][j]])
    # Assign value for internal node 7, based on internal node 5&6, and leaf node 4
    for j in range(0,vec_len):
        internal_node_7[j] = random.choice([internal_node_5[j], internal_node_6[j], 
                                           transformed_x[preorder[4]][j]])
    # append all node feature into an array
    transformed_x.append(internal_node_5)
    transformed_x.append(internal_node_6)
    transformed_x.append(internal_node_7)
    # create the node feature vector
    x = torch.tensor(transformed_x, dtype=torch.float)
    # Now we create the graph object as Data
    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_attr = edge_attr)
    return data

# File inputs

In [5]:
# get name of the script
# nameScript = sys.argv[0].split('/')[-1]
nameScript = "gae_model.py"
# get json file name of the script
nameJson = "gae.json"
# nameJson = sys.argv[1]
print("------------------------------------------------------------------------")
print("Training the Garph Auto Encoder for 5-taxa dataset")
print("------------------------------------------------------------------------")
print("Executing " + nameScript + " following " + nameJson, flush = True)

# opening Json file 
jsonFile = open(nameJson) 
dataJson = json.load(jsonFile)

# loading the input data from the json file
ngpu = dataJson["ngpu"]                  # number of GPUS
lr = dataJson["lr"]                      # learning rate
embedSize = dataJson["embedSize"]        # Embedding size
nEpochs = dataJson["nEpochs"]            # Number of Epochs
batchSize = dataJson["batchSize"]        # batchSize


data_root = dataJson["dataRoot"]         # data folder
model_root = dataJson["modelRoot"]       # folder to save the data

label_files = dataJson["labelFile"]      # file with labels
sequence_files = dataJson["matFile"]     # file with sequences
tree_files = dataJson["treeFile"]        # file with tree structure

if "summaryFile" in dataJson:
    summary_file = dataJson["summaryFile"]
else :
    summary_file = "summary_file.txt"


print("------------------------------------------------------------------------")
print("Loading Sequence Data in " + sequence_files, flush = True)
print("Loading Label Data in " + label_files, flush = True)
print("Loading Tree Data in " + tree_files, flush = True)

# we read the labels as list of strings
with open(data_root+label_files, 'r') as f:
    label_char = f.readlines()

# we read the sequence as a list of strings
with open(data_root+sequence_files, 'r') as f:
    seq_string = f.readlines()

with open(data_root+tree_files, 'r') as f:
    tree_newick = f.readlines()
    
n_samples = len(label_char)
seq_length = len(seq_string[0])-1
print("Number of samples:{}; Sequence length of each sample:{}"
        .format(n_samples, seq_length))
print("------------------------------------------------------------------------")

------------------------------------------------------------------------
Training the Garph Auto Encoder for 5-taxa dataset
------------------------------------------------------------------------
Executing gae_model.py following gae.json
------------------------------------------------------------------------
Loading Sequence Data in sequences12062021.in
Loading Label Data in labels12062021.in
Loading Tree Data in trees12062021.in
Number of samples:10000; Sequence length of each sample:1550
------------------------------------------------------------------------


# Data pre-processing

In [6]:
# We need to extract the dictionary with the relative positions
# for each aminoacid

# first we need to extract all the different chars
strL = ""
for c in seq_string[0][:-1]:
    if not c in strL:
        strL += c

# we sort them
strL = sorted(strL)

# we give them a relative order
dict_amino = {}
for ii, c in enumerate(strL):
    dict_amino[c] = ii

# looping over the labels and create array. Here each element of the
# label_char has the form "1\n", so we only take the first one
labels = np.fromiter(map(lambda x: int(x[0])-1,
                         label_char), dtype= np.int64)

In [53]:
# Create all graphs from raw dataset

dataset = []  # empty dataset for all graphs
# loop through all samples
for i in range(n_samples):
    # Get the ete tree format
    tree = tree_newick[i][:-1]
    t = Tree(tree)
    # get node feature, COO adjacency matrix, and edge feature
    data = construct_single_graph(i, t)
    # Validate if number of node and edges match
    if (not data.validate(raise_on_error=True)):
        print("Error! Node number and edge set does not match!")
        break
    # Add the graph into the dataset
    dataset.append(data)

# Model

In [82]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = TransformerConv(in_channels, 3 * out_channels, heads=4, concat=False, beta=True)
        self.conv_mu = TransformerConv(3 * out_channels, out_channels, heads=4, concat=False, beta=True)
        self.conv_logstd = TransformerConv(3 * out_channels, out_channels, heads=4, concat=False, beta=True)
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [83]:
in_channels = seq_length
out_channels = embedSize

model = VGAE(VariationalGCNEncoder(in_channels, out_channels))
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
model = model.to(device)

# Training

In [84]:
train_dataset = dataset[:9000]
test_dataset = dataset[9000:]
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [85]:
# Train
def run_one_epoch(data_loader):
    for _, batch in enumerate(tqdm(data_loader)):
        batch.to(device) 
        optimizer.zero_grad() 
        batch_neg_edge = batched_negative_sampling(batch.edge_index, batch.batch)
        z = model.encode(batch.x, batch.edge_index)
        loss = model.recon_loss(z, batch.edge_index, batch_neg_edge)
        loss = loss + (1 / batch.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
    return float(loss)

#def test(data_loader):
#    for _, batch in enumerate(tqdm(data_loader)):
#        with torch.no_grad():
#            z = model.encode(x, train_pos_edge_index)
#    return model.test(z, pos_edge_index, neg_edge_index)

In [86]:
for epoch in range(1, nEpochs+1):
    model.train()
    loss = run_one_epoch(train_loader)
    print(loss)

100%|█████████████████████████████████████████| 282/282 [00:47<00:00,  6.00it/s]


2.739959239959717


100%|█████████████████████████████████████████| 282/282 [01:07<00:00,  4.19it/s]


2.5902953147888184


100%|█████████████████████████████████████████| 282/282 [01:23<00:00,  3.38it/s]


...

2.5219616889953613


100%|█████████████████████████████████████████| 282/282 [01:13<00:00,  3.83it/s]


2.3720574378967285


100%|█████████████████████████████████████████| 282/282 [01:20<00:00,  3.52it/s]

2.489607810974121
